<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Nvidia Embeddings and Models

This notebook demonstrates how to set up a simple RAG pipeline using [NVIDIA AI Foundation Models](https://catalog.ngc.nvidia.com). At the end of this notebook, you will have a functioning Question/Answer pipeline that can answer questions using your supplied documents, powered by Astra DB, LangChain, and NVIDIA. 

## Prerequisites

You will need a vector-enabled Astra database and an NVIDIA NGC Account. 

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint: 
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`
* Create an [NVIDIA NGC Account](https://catalog.ngc.nvidia.com/)
  * Once signed in, navigate to `Catalog > AI Foundation Models > (Model)`
  * In the model page, select the `API` tab, then `Generate Key`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup
`ragstack-ai` includes all the packages you need to build a RAG pipeline. 

`langchain-nvidia-ai-endpoints` includes the NVIDIA models.

`datasets` is used to import a sample dataset.

In [1]:
! pip install -qU ragstack-ai langchain-nvidia-ai-endpoints datasets

In [2]:
import os
from getpass import getpass

# Enter your settings for Astra DB and NVidia:
os.environ["ASTRA_DB_ENDPOINT"] = input("Enter your Astra DB Endpoint: ")
os.environ["ASTRA_DB_TOKEN"] = getpass("Enter your Astra DB Token: ")
os.environ["NVIDIA_API_KEY"] = getpass("Enter your NVidia API Key: ")

In [4]:
# Collections are where documents are stored. ex: test
collection = "nvidia_test"

## Create RAG Pipeline

### Embedding Model and Vector Store

In [5]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedding = NVIDIAEmbeddings(model="nvolveqa_40k")

In [6]:
from langchain.vectorstores.astradb import AstraDB
import os

vstore = AstraDB(
    collection_name=collection,
    embedding=embedding,
    token=os.getenv("ASTRA_DB_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [12]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [8]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [9]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [ ]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection(collection).find())

### QA Retrieval

Retrieve context from your vector database, and pass it to the NVIDIA model with a prompt.

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_nvidia_ai_endpoints import ChatNVIDIA

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatNVIDIA(model="mixtral_8x7b")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subject are philosophers most concerned with?")

'In the given context, philosophers are most concerned with the subject of philosophy itself. Aristotle discusses philosophy as starting with wonder and also mentions that it can make people sick. Hegel, on the other hand, discusses the relationship between knowledge and education. Therefore, it can be inferred that philosophers are concerned with understanding the world, knowledge, and its implications on individuals and society.'

In [ ]:
# Add your questions here!
# chain.invoke("<your question>")

## Cleanup

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
# vstore.delete_collection()

You now have a functional RAG pipeline powered by NVIDIA! NVIDIA offers many different model types suited for different problems. Check out the [catalog](https://catalog.ngc.nvidia.com) for more. 